```
pip install --pre pystan
```

In [1]:
import nest_asyncio; nest_asyncio.apply()
import stan

import numpy as np

import jax

import numpyro; numpyro.enable_x64()
from numpyro.distributions import constraints, biject_to
from numpyro.infer import MCMC, NUTS

from age_model import model
from data import get_data, transform_data

In [2]:
data = get_data()
transformed_data = transform_data(data)
kernel = NUTS(model, step_size=0.02, max_tree_depth=15, target_accept_prob=0.95)
mcmc = MCMC(kernel, num_warmup=0, num_samples=1)
mcmc.run(np.zeros(2, dtype=np.uint32), transformed_data)
z = {k: v[0] for k, v in mcmc.get_samples().items()}

sample: 100%|██████████| 1/1 [00:26<00:00, 26.94s/it, 1 steps of size 2.00e-02. acc. prob=0.00]


In [3]:
with open("stan_model_test2.stan") as f:
    stan_code = f.read()
stan_model = stan.build(stan_code, data=data, random_seed=1)
stan_fit = stan_model.sample(num_chains=1, num_warmup=20, stepsize=1e-6, max_depth=5, num_samples=2)

Building...
Messages from stanc:
  The parameter impact_intv_children_effect has no priors.
  The parameter log_ifr_age_base has no priors.
  The parameter log_relsusceptibility_age_reduced has 2 priors.
  The parameter upswing_timeeff_reduced has 2 priors.
  Your Stan program has a parameter impact_intv_children_effect with a lower
  and upper bound in its declaration. These hard constraints are not
  recommended, for two reasons: (a) Except when there are logical or physical
  constraints, it is very unusual for you to be sure that a parameter will
  fall inside a specified range, and (b) The infinite gradient induced by a
  hard constraint can cause difficulties for Stan's sampling algorithm. As a
  consequence, we recommend soft constraints rather than hard constraints;
  for example, instead of constraining an elasticity parameter to fall
  between 0, and 1, leave it unconstrained and give it a normal(0.5,0.5)
  prior distribution.
Warning at '/tmp/httpstan_8bo7c2vp/model_636ufni4

In [4]:
params_support = {
  "R0": constraints.positive,
  "e_cases_N0": constraints.positive,
  "sd_dip_rnde": constraints.positive,
  "phi": constraints.positive,
  "log_ifr_age_base": constraints.less_than(0.),
  "hyper_log_ifr_age_rnde_mid1": constraints.positive,
  "hyper_log_ifr_age_rnde_mid2": constraints.positive,
  "hyper_log_ifr_age_rnde_old": constraints.positive,
  "log_ifr_age_rnde_mid1": constraints.positive,
  "log_ifr_age_rnde_mid2": constraints.positive,
  "log_ifr_age_rnde_old": constraints.positive,
  "upswing_timeeff_reduced": constraints.positive,
  "sd_upswing_timeeff_reduced": constraints.positive,
  "hyper_timeeff_shift_mid1": constraints.positive,
  "timeeff_shift_mid1": constraints.positive,
  "impact_intv_children_effect": constraints.interval(.1, 1.),
  "impact_intv_onlychildren_effect": constraints.positive
}

In [5]:
params = stan_fit.to_frame().to_dict()
z0 = {}
z1 = {}
for k in stan_fit.param_names:
    if k not in z:
        continue
    z0[k] = np.array([v[0] for k1, v in params.items() if k1.startswith(k)])
    z1[k] = np.array([v[1] for k1, v in params.items() if k1.startswith(k)])
    shape = z[k].shape
    if len(shape) == 2:
        z0[k] = z0[k].reshape((shape[1], shape[0])).T
        z1[k] = z1[k].reshape((shape[1], shape[0])).T
    elif len(shape) == 0:
        z0[k] = z0[k].reshape(())
        z1[k] = z1[k].reshape(())
    if k in params_support:
        z0[k] = biject_to(params_support[k]).inv(z0[k])
        z1[k] = biject_to(params_support[k]).inv(z1[k])
pe0 = -params["lp__"][0]
pe1 = -params["lp__"][1]
print("stan pe0 and pe1:", pe0, pe1)
print("stan pe diff:", pe1 - pe0)
potential_fn = mcmc.sampler._potential_fn_gen(transformed_data)
print("numpyro pe0 and pe1:", potential_fn(z0), potential_fn(z1))
print("numpyro pe diff:", potential_fn(z1) - potential_fn(z0))

stan pe0 and pe1: 45829.99606673507 45827.932293768274
stan pe diff: -2.063772966794204
numpyro pe0 and pe1: 45829.996066735024 45827.93229376833
numpyro pe diff: -2.0637729666923406


In [6]:
data["smoothed_logcases_week_map"]

array([[[ 29,  30,  31,  32,  33,  34,  35],
        [ 36,  37,  38,  39,  40,  41,  42],
        [ 43,  44,  45,  46,  47,  48,  49],
        [ 50,  51,  52,  53,  54,  55,  56],
        [ 57,  58,  59,  60,  61,  62,  63],
        [ 64,  65,  66,  67,  68,  69,  70],
        [ 71,  72,  73,  74,  75,  76,  77],
        [ 78,  79,  80,  81,  82,  83,  84],
        [ 85,  86,  87,  88,  89,  90,  91],
        [ 92,  93,  94,  95,  96,  97,  98],
        [ 99, 100, 101, 102, 103, 104, 105],
        [106, 107, 108, 109, 110, 111, 112],
        [113, 114, 115, 116, 117, 118, 119],
        [120, 121, 122, 123, 124, 125, 126],
        [127, 128, 129, 130, 131, 132, 133],
        [134, 135, 136, 137, 138, 139, 140],
        [141, 142, 143, 144, 145, 146, 147],
        [148, 149, 150, 151, 152, 153, 154],
        [155, 156, 157, 158, 159, 160, 161],
        [162, 163, 164, 165, 166, 167, 168],
        [169, 170, 171, 172, 173, 174, 175],
        [176, 177, 178, 179, 180, 181, 182],
        [1

stan pe0 and pe1: 45829.99606673507 45827.932293768274
stan pe diff: -2.063772966794204
numpyro pe0 and pe1: 45881.68816239705 45879.59112529917
numpyro pe diff: -2.0970370978757273

In [7]:
data["school_case_time_idx"]

array([[ -1,  -1],
       [ -1,  -1],
       [201, 249],
       [ -1,  -1]])

In [8]:
pe_and_grad = jax.jit(jax.value_and_grad(potential_fn))
print(pe_and_grad(z1)[0])

45827.93229376833


In [9]:
%%time
a, b = pe_and_grad(z0)
print(a)

45829.99606673503
CPU times: user 31.8 ms, sys: 153 µs, total: 31.9 ms
Wall time: 22.1 ms
